This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'

res = requests.get(f'https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}\?limit=1&api_key={API_KEY}')
res.text

'{\n  "quandl_error": {\n    "code": "QESx05",\n    "message": "You have requested an unknown format. Please check the format you requested and try again."\n  }\n}'

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = res.json()
json_data

{'quandl_error': {'code': 'QESx05',
  'message': 'You have requested an unknown format. Please check the format you requested and try again.'}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
# 1. Collect AFX_X data for all of 2017
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'

res_2017 = requests.get(f'https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}')

# 2. Convert to dictionary
json_2017 = res_2017.json()
type(json_2017)

dict

In [7]:
data_2017 = json_2017['dataset_data']['data']
column_names = json_2017['dataset_data']['column_names']
column_names

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [8]:
# 3. Calculate highest and lowest opening prices in 2017. 

opening = [row[1] for row in data_2017 if row[1] is not None]

max_open = max(opening)
min_open = min(opening)

print(f'Highest 2017 opening price: {max_open:.2f}')
print(f'Lowest 2017 opening price: {min_open:.2f}')


Highest 2017 opening price: 53.11
Lowest 2017 opening price: 34.00


In [9]:
# 4. Largest change in one day (High - Low). 
intra_day_diff = [abs(row[2] - row[3]) for row in data_2017 if row[2] is not None or row[3] is not None]

max_diff = max(intra_day_diff)

print(f'Largest intraday change: {max_diff:.2f}')

Largest intraday change: 2.81


In [10]:
# 5. Largest change between two days (Close[x] - Close[x-1]). 
daily_diff = []

for i, row in enumerate(data_2017):
        if i == 0:
            daily_diff.append(0)
        else: 
            daily_diff.append(abs(row[4] - data_2017[i-1][4]))
            
max_daily_diff = max(daily_diff)

print(f'Largest daily change: {max_daily_diff:.2f}')

Largest daily change: 2.56


In [11]:
#6. Average daily trading volume. 
volume = [row[6] for row in data_2017 if row[6] is not None]

mean_vol = sum(volume) / len(volume)

print(f'Average daily trading volume: {mean_vol:.0f}')

Average daily trading volume: 89124


In [12]:
# 7. Median daily trading volume. 
volume_sort = sorted(volume)

if len(volume_sort)%2 == 1:
    index = int((len(volume_sort) + 1)/2)
    median_vol = volume_sort[index-1]
else:
    index = int((len(volume_sort))/2)
    median_vol = volume_sort[index-1]

print(f'Median daily trading volume: {median_vol:.0f}')

Median daily trading volume: 76286
